### Develop notebook

**Todos**

- [x] Sanity check implementation of IPD where agent two sees the actions of agent one
- [ ] Test functions and visualize Q-value trajectories

In [138]:
%load_ext autoreload
%autoreload 2

from typing import Dict, List, Tuple, Union
from gym import Space, spaces
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from prisoners_dilemma import utils
from prisoners_dilemma.env import PrisonersDilemmaEnv

sns.set('notebook', font_scale=1.1, rc={'figure.figsize': (7, 4)})
sns.set_style('ticks', rc={'figure.facecolor': 'none', 'axes.facecolor': 'none'})
matplotlib.rcParams['figure.facecolor'] = 'white'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
# Setup config
config = {
    'num_simuls': 10,
    'num_episodes': 1000,
    'num_agents': 2,
    'num_actions': 2,
    'verbose': False,
    'init_type':'zeros',
}

config['payoffs'] = {
    'reward_payoff': 2,
    'tempta_payoff': 3,
    'sucker_payoff': 0,
    'punish_payoff': 1,
}

config['params'] = {
    'alpha': np.array([0.1, 0.1]),
    'eps': np.array([0.1, 0.1]),
    'gamma': np.array([0.5, 0.5]),
}

# Create arrays
q_traj_one = np.zeros((config["num_simuls"], config["num_episodes"], config["num_actions"]))
q_traj_two = np.zeros((config["num_simuls"], config["num_episodes"], config["num_actions"], config["num_actions"]))
rewards_seq = np.zeros((config["num_simuls"], config["num_episodes"], config["num_agents"]))
action_seq = np.zeros((config["num_simuls"], config["num_episodes"], config["num_agents"]), dtype=int)

for simul_i in range(config['num_simuls']):
    q_traj_one[simul_i, :, :], q_traj_two[simul_i, :, :], rewards_seq[simul_i, :, :], action_seq[simul_i, :, :] = utils.run_extended_ipd_exp(config)